In [ ]:
from dataclasses import asdict
from pydantic.dataclasses import dataclass
from pydantic import parse_obj_as, Field, BaseModel

from my_scientific_profile.semantic_scholar.authors import (
    get_author_info
)
from my_scientific_profile.papers.papers import fetch_paper_info
from my_scientific_profile.crossref.works import get_crossref_work_by_doi
import pandas as pd
from my_scientific_profile.authors.authors import convert_orcid_author_to_author, get_author_from_orcid_or_crossref, Author
from my_scientific_profile.orcid.authors import (
    search_for_author_by_name, search_for_author_by_orcid_id, OrcidAuthor
)
from my_scientific_profile.orcid.employments import (
    fetch_employment_for_orcid_id, 
)
from my_scientific_profile.orcid.works import get_put_code_to_doi_map
from my_scientific_profile.orcid.utils import get_orcid_query

In [ ]:
doi1 = "10.1140/epjst/e2016-60114-5"
doi2 = "10.1063/1.4941455"

In [ ]:
paper1 = fetch_paper_info(doi1)
paper2 = fetch_paper_info(doi2)

In [ ]:
paper1.authors

In [ ]:
paper2.authors

In [ ]:
list(set(*[paper1.authors + paper2.authors]))

In [ ]:
Author._instances

In [ ]:
dois = list(set(list(get_put_code_to_doi_map().values())))

In [ ]:
work = get_crossref_work_by_doi(dois[0])
work.message

In [ ]:
work.message.author

In [ ]:
authors = [get_author_from_orcid_or_crossref(author) for doi in dois for author in get_crossref_work_by_doi(doi).message.author]
authors

In [ ]:
df = pd.json_normalize([asdict(author) for author in authors])
df

In [ ]:
df_sub = df[["given", "family", "orcid", "affiliation.name", "affiliation.city", "affiliation.country", "email"]].groupby(by=["given", "family"]).first().sort_values("family")
df_sub = df_sub.reset_index()
df_sub['initial'] = df_sub.given.str[0]
df_sub

In [ ]:
df_wo_orcid = df_sub.loc[pd.isna(df_sub.orcid)].sort_values('family', ignore_index=True)
df_wo_orcid['partial_name'] = df_wo_orcid['initial'] + " " + df_wo_orcid['family']
df_wo_orcid['full_name'] = df_wo_orcid['given'] + " " + df_wo_orcid['family']
df_wo_orcid.head(40)

In [ ]:
df_w_orcid = df_sub.loc[pd.notna(df_sub.orcid)].sort_values('family', ignore_index=True)
df_w_orcid['partial_name'] = df_w_orcid['initial'] + " " + df_w_orcid['family']
df_w_orcid['full_name'] = df_w_orcid['given'] + " " + df_w_orcid['family']
df_w_orcid

In [ ]:
pd.merge(df_wo_orcid, df_w_orcid, on='partial_name')

In [ ]:
get_orcid_query(
    "expanded-search", 
    orcid_id=None, 
    suffix=f"?q=orcid:0000-0002-4003-2272"
)

In [ ]:
orcid_search = search_for_author_by_name("Dirk", "Schneider")
orcid_search

In [ ]:
orcid_search = search_for_author_by_orcid_id("0000-0002-4003-2272")
orcid_search

In [ ]:
orcid_search[0].employment

In [ ]:
fetch_employment_for_orcid_id("0000-0001-7930-8806")

In [ ]:
orcid_search[0].employment.affiliation_group[0].summaries[0].employment_summary.organization.name

In [ ]:
convert_orcid_author_to_author(orcid_search[0])

In [ ]:
q = get_orcid_query("employments", orcid_id="0000-0003-4837-3888", suffix=None)
q

In [ ]:
q['affiliation-group'][0]["summaries"][0]["employment-summary"]